<a href="https://colab.research.google.com/github/Aligreu/Deep-Learning-with-Tensorflow-Projects/blob/main/trainPyTorchModelwithDiabetesDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [4]:
torch.cuda.is_available()

True

In [8]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: diabetes.csv            


In [9]:
class DiabetesDataset(Dataset):
    def __init__(self, path):
        super().__init__()
        df = pd.read_csv(path)

        self.data = df.to_numpy()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        features, label = self.data[index, :-1], self.data[index, -1]
        return features, label

data = DiabetesDataset("/content/diabetes.csv")

In [10]:
train_data = DataLoader(data,
                        batch_size=8,
                        shuffle=True)

In [12]:
features, target = next(iter(train_data))

In [14]:
from torch import nn as nn
import torch.nn.functional as F
from torch import optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))

        return x


device = "cuda" if torch.cuda.is_available() else "cpu"
model = Net().to(device)

In [36]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for i in range(100):
    for features, target in train_data:
        optimizer.zero_grad()
        output = model(features.to(torch.float32).to(device))
        target = target.float()
        loss = criterion(output.to(device), target.view(-1,1).to(device))
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(f"The Loss is {loss:.2f}")

The Loss is 0.72
The Loss is 0.62
The Loss is 0.35
The Loss is 0.24
The Loss is 0.28
The Loss is 0.57
The Loss is 0.40
The Loss is 0.50
The Loss is 0.19
The Loss is 0.41


In [20]:
!pip install torchmetrics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 6.7 MB/s eta 0:00:00


In [37]:
from torchmetrics import Accuracy

acc = Accuracy(task="binary").to(device)

model.eval()

with torch.no_grad():
    for features, target in train_data:
        output = model(features.to(torch.float32).to(device))
        preds = (output > 0.5).float().to(device)
        acc(preds.to(device), target.view(-1, 1).to(device))

test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")


Test accuracy: 0.7604166865348816
